<a href="https://colab.research.google.com/github/hatahataDev/SDGsproject/blob/master/SDGs_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#入力された文章がSDGsの17の目標にそれぞれどのくらい関連しているのかを表示するプログラム
data_dir="学習データを入れる。"" #CSVファイルのパス
"""
入力用データ成型機
"""
#https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0 参考記事
#https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0
#課題～ハイパーパラメータをいじる、あるいはデータの抜き出し、整形の制度を上げる。
#数値のベクトル化のところのアルゴリズムが少し不自然な気がするからそこの修正
#ネット回線ないとコード動きません。
#ネットのSDGs用語も辞書に足し合わせるべし←ネットに乗っている用語は重要度を上げるなどの改良の余地あり。
#大文字を小文字にする処理や記号を抜き取る処理を追加する。
import os
import csv
import nltk
import numpy as np
import keras
import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split ### 追加


nltk.download('punkt') #nltkを使うためのソフト群
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

csv_file = open(data_dir,'r') #学習データ
p_n_list = input("予測したい文章を入力してください。")
#p_csv_file= open(predict_dir,"r") #予測したいデータ
"""
csvファイルは左から目標番号、タイトル、アブスト、結論の順で入れること。
"""
p_list,t_list,a_list = [],[],[]
#c_list= []

delete= ["a","%","of","in","on","to","for","are","at"] #関連性のない語を集めた配列

#p_n_list=[]#予測したいデータのテキストバッグ

#for row in csv.reader(p_csv_file):
#  p_n_list.append(row[1])
#  p_n_list.append(row[2])

x, y = [], []
count = 0

for row in csv.reader(csv_file):
  #以下リストを作成
    p_list.append(row[0]) #取得したい列番号を指定（0始まり）#目標番号をすべて取得
    t_list.append(row[1]) #タイトルをすべて取得
    a_list.append(row[2]) #アブストをすべて取得
    #c_list.append(row[3]) #結論も入れる

del p_list[0] #0番目を削除
del t_list[0] #0番目は削除
del a_list[0] #0番目は削除。これで_list[i]でそれぞれの列を抜き出すことができる。
#del c_list[0]

#以下３行　予測したいデータの形態素解析
#p_n_list=p_n_list[0]+p_n_list[1] 
p_n_words = nltk.word_tokenize(p_n_list)
p_n_wards=nltk.pos_tag(p_n_words)#[("単語","品詞")....]の２次元リストをここで生成


p_result = [] #名詞の数字配列を入れるp_resultを定義
p_N_dic = [] #名詞のstring型配列を定義
#以下名詞を抜き出しN_dicに入れ込み
for word,part in p_n_wards: #wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower()
        p_N_dic.append(sword)

dic={} #辞書で定義
for i,p in enumerate(p_list): #p_listのiをカウンタ変数,ｐを正解ラベル
  j=i+1 #p_list,a_listなどは0番目が存在しないため無理やりjをカウンター変数に置き換え
  if j>=len(p_list): #+1してるから最後の一つだけ存在しないjが入るためここで分岐処理　t_list[j_max]は存在しないためエラー
    break
  N_dic=[] #ここで名詞リストは初期化
  #print(t_list[i])
  t_string = t_list[j]
  t_words = nltk.word_tokenize(t_string)
  t_wards=nltk.pos_tag(t_words)#[("単語","品詞")....]の２次元リストをここで生成
  #print(t_wards)
  a_string = a_list[j]
  a_words = nltk.word_tokenize(a_string)
  a_wards=nltk.pos_tag(a_words)
  """
  c_string = c_list[j]
  c_words = nltk.word_tokenize(c_string)
  c_wards=nltk.pos_tag(c_words)
  """
  for word,part in t_wards: #wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower() #小文字に変換
        N_dic.append(sword)
  for word,part in a_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete:
        sword=word.lower()
        N_dic.append(sword)  
  """
  for word,part in c_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN": #以下if文は名詞のみをN_dicというリストに格納
      N_dic.append(word)
    if part=="NNP":
      N_dic.append(word)
  """
  print(N_dic)#すべてN_dicにまとめてしまう。

  result=[]
  for word in N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    #print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
  x.append(result)  # リストを配列 ｘ に追加 x = [ [ 10, 11, 12, 3, 8, 13,  … 38 ] , [ 39, 40, 12, 16, 19, … ,56 ] , ….  ,  [ 1200, 1504, 3, 15, … 3300 ] ] 
  p_p=int(p)  #pは文字型で入っているのでキャスト
  y.append(p_p)  # 正解ラベルを配列 y に追加 y = [ 0, 0, 0, 0, … ,8, 8, 8, 8 ] 

#以下のfor 予測したいデータの名詞配列の数値化 or 辞書の生成
for word in p_N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    #print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    p_result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している

print(x)
print(y)



max_words = len(dic)
batch_size = 150
epochs = 30

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
 
num_classes = np.max(y_train) +1
print(num_classes, 'classes')
 
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
 
print('Convert class vector to binary class matrix '
'(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
 
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))#単層ニューラルネットワーク

 
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']) #損失関数を変更している。
 
history = model.fit(x_train, y_train,
batch_size=batch_size,
epochs=epochs,
verbose=1,
validation_split=0.1)
score = model.evaluate(x_test, y_test,
batch_size=batch_size, verbose=1)

pre=model.predict( x_test, batch_size=batch_size, verbose=1) #それぞれの正解ラベルに対する確率を返す

#予測データのバイナリー化
arr=np.zeros(len(dic)) #辞書数分のバイナリー配列を生成。すべて０で最初は生成
for f in p_result: #予測したい文脈の名詞の番号のところに１を立てる。
  arr[f]=1

arr=np.array(arr) #numpy配列に変換
arr.reshape(1,len(dic))
arrd=np.stack([arr, arr]) #無理やり配列を拡張。model.predictの引数に拡張しないと入れられないため。(2,len(dic))の配列にする

#print(arrd.shape)
pre1=model.predict( arrd, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])
print(pre1)

#出力コード
result= []
for i,c in enumerate(pre1[0]): #２次元配列を生成
  hum=[c,i]
  result.append(hum)
result2=sorted(result,reverse=True) #大きい順にソート

j=0
for k,l in enumerate(result2):
  if l[1]==0:
    j=1
    continue
  if j==1:
    print(str(k)+"番目に関連が高いのは"+str(l[1])+"の目標で,関連度は"+str(l[0]*100)+"％です。")
  else:
    print(str(k+1)+"番目に関連が高いのは"+str(l[1])+"の目標で,関連度は"+str(l[0]*100)+"％です")

### Plot accuracy
import matplotlib.pyplot as plt
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(1, len(acc) + 1)
 
plt.plot(epochs, acc, "bo", label = "Training acc" )
plt.plot(epochs, val_acc, "b", label = "Validation acc")
plt.title("Training and Validation accuracy")
plt.legend()
plt.savefig("acc.png")
plt.close()
 
### plot Confusion Matrix
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix
 
def print_cmx(y_true, y_pred):
  labels = sorted(list(set(y_true)))
  cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
 
  df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
  plt.figure(figsize = (10,7))
  sn.heatmap(df_cmx, annot=True, fmt="d") ### ヒートマップの表示仕様
  plt.title("Confusion Matrix")
  plt.xlabel("predict_classes")
  plt.ylabel("true_classes")
  plt.savefig("c_matrix.png")
  plt.close()
 
predict_classes = model.predict_classes(x_test[1:10000,], batch_size=32) ### 予測したラベルを取得
true_classes = np.argmax(y_test[1:10000],1) ### 実際のラベルを取得
print(confusion_matrix(true_classes, predict_classes))
print_cmx(true_classes, predict_classes)

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
予測したい文章を入力してください。By 2030, ensure that all young people and the majority (both men and women) adults have literacy and basic computing skills. By 2030, education for sustainable development and sustainable lifestyles, human rights, gender equality, promotion of peace and non-violent culture, global citizenship, sustainable development of cultural diversity and culture Ensure that all learners have the knowledge and skills necessary to promote sustainable development through education in understanding their contributions. Establish and improve educational facilities that are child, disability and gender sensitive and provid